# Project Zillow

- Predict the tax assessed property value of Zillow Single Family Residential properties with transaction dates in 2017.


# Goal

In [ ]:
- Identify drivers of property value.
- Use drivers to develop a machine learning model to predict Single Family Residential property value.

# Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, PolynomialFeatures
# local
import wrangle as w
import explore as e
import modeling as m

# Acquire

- Data acquired from Codeup MySQL DB
- Data initially acquired on 7 May 2023
- It contained 52,441 rows and 62 columns before cleaning
- Each row represents a unique parcel for each transaction it had in 2017
- Each column represents an element of the parcel

# Prepare

**Prepare Actions:**

Replaced null values with appropriate values if logical
0 for most columns

lotsize = area if lotsize value is missing

Split transaction date to month and day since all are 2017

Renamed columns to promote readability

Removed columns that did not contain useful information

too many null values that couldn't be logically imputed

related to target (property value)

duplicated information

Removed rows

nulls that couldn't logically be imputed

duplicate parcels, kept most recent transaction date

Checked that column data types were appropriate

Handled outliers

property value < 98th quantile

area < 6000

Mapped FIPS code to county

Encoded categorical variables

created dummy variables for categorical variables

Split data into train, validate and test (approx. 60/20/20)

Scaled using Standard Scaler for modeling